<a href="https://colab.research.google.com/github/Aman-coder-debug/skin_disease_detector/blob/main/Askindect.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# ✅ Install required packages
!pip install -q gdown gradio tensorflow

# ✅ Download ZIP file from Google Drive
import gdown
zip_file_id = "11lS4A06C4-3kiwbfya7o8072ymyaoWn9"
gdown.download(f"https://drive.google.com/uc?id={zip_file_id}", "skin_dataset.zip", quiet=False)

# ✅ Unzip the downloaded file
import zipfile
with zipfile.ZipFile("skin_dataset.zip", "r") as zip_ref:
    zip_ref.extractall("skin_data")

# ✅ Load the extracted CSV
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

csv_path = "skin_data/hmnist_28_28_RGB.csv"  # Adjust if inside a folder
df = pd.read_csv(csv_path)

X = df.drop('label', axis=1).values
y = df['label'].values

# ✅ Preprocess image and labels
X = X.reshape(-1, 28, 28, 3).astype('float32') / 255.0
y = to_categorical(y, num_classes=7)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# ✅ Build CNN Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(7, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=5, batch_size=64, validation_split=0.1)

# ✅ Gradio Interface
import gradio as gr
from PIL import Image

class_names = [
    "Actinic keratoses", "Basal cell carcinoma", "Benign keratosis-like lesions",
    "Dermatofibroma", "Melanocytic nevi", "Vascular lesions", "Melanoma"
]

precautions = {
    0: "Use sunscreen, avoid direct sunlight between 10am-4pm.",
    1: "Visit dermatologist. Early removal often helps.",
    2: "Monitor for changes, moisturize skin regularly.",
    3: "Usually harmless but monitor growth.",
    4: "Usually benign. Still, monitor for any color/size changes.",
    5: "Seek immediate medical advice.",
    6: "High risk. See an oncologist or skin specialist immediately."
}

def predict_image(img):
    img = img.resize((28, 28)).convert('RGB')
    img = np.array(img).reshape(1, 28, 28, 3).astype('float32') / 255.0
    pred = model.predict(img)[0]
    class_idx = np.argmax(pred)
    return {
        "Prediction": class_names[class_idx],
        "Precaution": precautions[class_idx]
    }

iface = gr.Interface(
    fn=predict_image,
    inputs=gr.Image(type="pil", label="Upload Skin Image"),
    outputs=[
        gr.Textbox(label="Predicted Disease"),
        gr.Textbox(label="Suggested Precaution")
    ],
    title="Askindect - Skin Disease Predictor",
    description="Upload a 28x28 RGB skin lesion image to get a prediction and safety tips."
)

iface.launch()


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.2/54.2 MB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.1/323.1 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 25.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 3.6 MB/s eta 0:00:00


Downloading...
From (original): https://drive.google.com/uc?id=11lS4A06C4-3kiwbfya7o8072ymyaoWn9
From (redirected): https://drive.google.com/uc?id=11lS4A06C4-3kiwbfya7o8072ymyaoWn9&confirm=t&uuid=0f01751d-07f8-49e4-b464-5d9d00e7c8b6
To: /content/skin_dataset.zip
100%|██████████| 27.5M/27.5M [00:00<00:00, 45.6MB/s]
/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/5
113/113 ━━━━━━━━━━━━━━━━━━━━ 10s 74ms/step - accuracy: 0.6490 - loss: 1.2079 - val_accuracy: 0.6958 - val_loss: 0.9300
Epoch 2/5
113/113 ━━━━━━━━━━━━━━━━━━━━ 7s 57ms/step - accuracy: 0.6668 - loss: 0.9947 - val_accuracy: 0.6883 - val_loss: 0.8709
Epoch 3/5
113/113 ━━━━━━━━━━━━━━━━━━━━ 10s 58ms/step - accuracy: 0.6716 - loss: 0.9381 - val_accuracy: 0.6958 - val_loss: 0.8191
Epoch 4/5
113/113 ━━━━━━━━━━━━━━━━━━━━ 8s 71ms/step - accuracy: 0.6769 - loss: 0.9078 - val_accuracy: 0.7020 - val_loss: 0.7992
Epoch 5/5
113/113 ━━━━━━━━━━━━━━━━━━━━ 7s 59ms/step - accuracy: 0.6874 - loss: 0.8707 - val_accuracy: 0.7070 - val_loss: 0.8310
It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://3278aa